<h2>Baseline Code</h2>

In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model, to_categorical
import warnings
import re

In [2]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-11-04  오후 11:05    <DIR>          .
2020-11-04  오후 11:05    <DIR>          ..
2020-11-02  오전 10:16    <DIR>          .ipynb_checkpoints
2020-10-29  오후 06:12         3,959,848 NPL_base_5layer.h5
2020-11-04  오후 11:05            41,958 NPL_BaseLine.ipynb
2020-11-03  오후 04:25            33,918 NPL_RNN1.ipynb
2020-10-29  오후 04:43    <DIR>          open
2020-10-29  오후 02:41         9,263,779 open.zip
               4개 파일          13,299,503 바이트
               4개 디렉터리  77,436,153,856 바이트 남음


In [3]:
#파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [4]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [5]:
train.author.describe()

count    54879.000000
mean         1.944988
std          1.391632
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: author, dtype: float64

<h2>전처리

In [6]:
#부호를 제거해주는 함수

def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [7]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


1. 불용어 제거 remove_stopwords

In [8]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

2. One-Hot EnCoding

3. N-grams

4. Bag-of-words

In [9]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
#train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
#test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [10]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [11]:
train
#문체를 다 짤라먹었는데 왜 다 짤라먹었을까????
# , 활용 "" 활용같은것도 중요할것 같은데

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands d...,3
...,...,...,...
54874,54874,is that you mr smith odin whispered i hardly d...,2
54875,54875,i told my plan to the captain and between us w...,4
54876,54876,your sincere wellwisher friend and sister luc...,1
54877,54877,then you wanted me to lend you money,3


<h2>모델링

In [12]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [13]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [14]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

<h3> 가벼운 NLP모델 생성

In [15]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [16]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


<h3> 모델 fit

In [17]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 5s - loss: 1.5409 - accuracy: 0.2986 - val_loss: 1.4610 - val_accuracy: 0.3606
Epoch 2/20
1372/1372 - 5s - loss: 1.3397 - accuracy: 0.4424 - val_loss: 1.2350 - val_accuracy: 0.5045
Epoch 3/20
1372/1372 - 4s - loss: 1.1466 - accuracy: 0.5400 - val_loss: 1.1127 - val_accuracy: 0.5510
Epoch 4/20
1372/1372 - 5s - loss: 1.0298 - accuracy: 0.5865 - val_loss: 1.0193 - val_accuracy: 0.5907
Epoch 5/20
1372/1372 - 5s - loss: 0.9455 - accuracy: 0.6212 - val_loss: 0.9731 - val_accuracy: 0.6163
Epoch 6/20
1372/1372 - 5s - loss: 0.8751 - accuracy: 0.6528 - val_loss: 0.9111 - val_accuracy: 0.6452
Epoch 7/20
1372/1372 - 5s - loss: 0.8157 - accuracy: 0.6795 - val_loss: 0.8915 - val_accuracy: 0.6535
Epoch 8/20
1372/1372 - 4s - loss: 0.7668 - accuracy: 0.7043 - val_loss: 0.8468 - val_accuracy: 0.6767
Epoch 9/20
1372/1372 - 5s - loss: 0.7216 - accuracy: 0.7259 - val_loss: 0.8271 - val_accuracy: 0.6859
Epoch 10/20
1372/1372 - 4s - loss: 0.6832 - accuracy: 0.7430 - val_loss: 0.8185 - 

In [18]:
# predict values
pred = model.predict_proba(test_padded)
pred

Instructions for updating:
Please use `model.predict()` instead.


array([[6.4656168e-05, 3.9847305e-01, 2.1663062e-01, 3.8459781e-01,
        2.3381431e-04],
       [2.5241068e-01, 1.2716509e-01, 3.7477821e-01, 6.5152002e-03,
        2.3913088e-01],
       [9.9354184e-01, 2.3465471e-03, 7.1884324e-06, 7.4180463e-08,
        4.1042366e-03],
       ...,
       [1.8457332e-04, 9.9966991e-01, 7.0698628e-11, 1.4549521e-04,
        4.0610829e-10],
       [2.8258661e-04, 9.9967813e-01, 4.8052198e-07, 3.6447611e-05,
        2.3873072e-06],
       [9.7068864e-01, 3.6160491e-05, 4.1344141e-05, 1.8477074e-07,
        2.9233692e-02]], dtype=float32)

<h3> Submission

In [19]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,6.465617e-05,3.984731e-01,2.166306e-01,3.845978e-01,2.338143e-04
1,1,2.524107e-01,1.271651e-01,3.747782e-01,6.515200e-03,2.391309e-01
2,2,9.935418e-01,2.346547e-03,7.188432e-06,7.418046e-08,4.104237e-03
3,3,3.425989e-07,1.740132e-12,9.991981e-01,1.058132e-09,8.014572e-04
4,4,9.987670e-01,4.193552e-04,1.112029e-04,2.485261e-04,4.539908e-04
...,...,...,...,...,...,...
19612,19612,4.348089e-06,9.999957e-01,1.348548e-18,2.638760e-09,1.869285e-16
19613,19613,5.823138e-02,1.259564e-05,1.837534e-03,7.681488e-10,9.399185e-01
19614,19614,1.845733e-04,9.996699e-01,7.069863e-11,1.454952e-04,4.061083e-10
19615,19615,2.825866e-04,9.996781e-01,4.805220e-07,3.644761e-05,2.387307e-06


In [20]:
sample_submission.to_csv('open/sub/submission_BASELINE.csv', index = False, encoding = 'utf-8')
